In [1]:
import sourcecode.Text.generate
import $ivy.`org.apache.spark::spark-sql:3.5.2`
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SQLContext
import java.nio.file.attribute.BasicFileAttributes
import java.nio.file._
import scala.collection.mutable.ArrayBuffer

import sourcecode.Text.generate
import $ivy.$
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SQLContext
import java.nio.file.attribute.BasicFileAttributes
import java.nio.file._
import scala.collection.mutable.ArrayBuffer

In [2]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import org.apache.spark.SparkConf
val spark = SparkSession.builder.master("spark://spark:7077").config("spark.executor.memory", "10g").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
val sqlContext = spark.sqlContext
import sqlContext.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/12/13 12:39:05 INFO SparkContext: Running Spark version 3.5.2
24/12/13 12:39:05 INFO SparkContext: OS info Linux, 6.8.0-50-generic, amd64
24/12/13 12:39:05 INFO SparkContext: Java version 1.8.0_402
24/12/13 12:39:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/13 12:39:05 INFO ResourceUtils: ==============================================================
24/12/13 12:39:05 INFO ResourceUtils: No custom resources configured for spark.driver.
24/12/13 12:39:05 INFO ResourceUtils: ==============================================================
24/12/13 12:39:05 INFO SparkContext: Submitted application: 428eb541-8d12-4714-ae6b-661adb9a0d61
24/12/13 12:39:05 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(memory -> name: memory, amount: 10240, script: , vendor: , offHeap -> name: offHeap, amount:

import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import org.apache.spark.SparkConf
spark: SparkSession = org.apache.spark.sql.SparkSession@ba38bfa
sqlContext: SQLContext = org.apache.spark.sql.SQLContext@482cd4e5
import sqlContext.implicits._

In [3]:
def search_files(path: String, extension: String): ArrayBuffer[Path] = {
  val files = ArrayBuffer.empty[Path]
  val root = Paths.get(path)

  Files.walkFileTree(root, new SimpleFileVisitor[Path] {
    override def visitFile(file: Path, attrs: BasicFileAttributes) = {
      if (file.getFileName.toString.endsWith(extension)) {
        files += file
      }
      FileVisitResult.CONTINUE
    }
  })

  files
}

defined function search_files

In [4]:
// Load parquet files into a DataFrame array
val df_array = search_files("/main/data/source_as_parquet", ".parquet")
  .map { f =>
    spark.read
      .format("parquet")
      .load(f.toString)
  }
// Concat all
val df = df_array.reduce((df1, df2) => df1.unionByName(df2, allowMissingColumns=true))
df.createOrReplaceTempView("df")
df.show(5)

+-----------+-------------------+-----------------+---+-------------------+----+-----------+-----+----------+---------------+-------------------+--------------------+--------------------+--------------------+-----+---------------------+-----------+-------+--------------------+--------------------+-----------------+------------+
|ResoTrigger|               Time|         Position|Lap|           Velocity|Pump|optotrigger|Licks|Licking_MM|Position_binned|Position_binned_2cm|1_cm_binned_position|2_cm_binned_position|5_cm_binned_position|Lap_2|Stimulation condition|Stimulation|Zone ID|           file_name|         neural_data|__index_level_0__|Time_counter|
+-----------+-------------------+-----------------+---+-------------------+----+-----------+-----+----------+---------------+-------------------+--------------------+--------------------+--------------------+-----+---------------------+-----------+-------+--------------------+--------------------+-----------------+------------+
|         

df_array: ArrayBuffer[org.apache.spark.sql.package.DataFrame] = ArrayBuffer(
  [ResoTrigger: bigint, Time: double ... 19 more fields],
  [ResoTrigger: bigint, Time: double ... 19 more fields],
  [ResoTrigger: bigint, Time: double ... 19 more fields],
  [ResoTrigger: bigint, Time: double ... 19 more fields],
  [ResoTrigger: bigint, Time: double ... 19 more fields],
  [ResoTrigger: bigint, Time: double ... 19 more fields],
  [ResoTrigger: bigint, Time: double ... 20 more fields],
  [ResoTrigger: bigint, Time: double ... 20 more fields],
  [ResoTrigger: bigint, Time: double ... 20 more fields],
  [ResoTrigger: bigint, Time: double ... 19 more fields],
  [ResoTrigger: bigint, Time: double ... 20 more fields],
  [ResoTrigger: bigint, Time: double ... 17 more fields],
  [ResoTrigger: bigint, Time: double ... 20 more fields],
  [ResoTrigger: bigint, Time: double ... 20 more fields],
  [ResoTrigger: bigint, Time: double ... 20 more fields],
  [ResoTrigger: bigint, Time: double ... 20 more fiel

In [5]:
val windowSpec = Window.partitionBy("file_name").orderBy("time")
val dv = df.select(
      $"Time".as("time"),
      $"Position".as("position"),
      $"Velocity".as("original_velocity"),
      $"Pump".as("pump"),
      $"neural_data",
      $"file_name"
)
.withColumn("dt", 
  $"time" - when(
    (lag("time", 1).over(windowSpec)).isNull, 
    0
  ).otherwise(
    lag("time", 1).over(windowSpec)
  )
)
.withColumn("dx",
  $"position" - when(
    (lag("position", 1).over(windowSpec)).isNull, 
    0
  ).otherwise(
    lag("position", 1).over(windowSpec)
  )
)
.withColumn("velocity",
  when(
    $"dt" === 0, 
    0
  ).otherwise(
    $"dx" / $"dt"
  )
)
.na.drop()
.where(!($"pump" < 1 && $"pump" > 0))
.where("exists(neural_data, x -> x is not null)")
dv.createOrReplaceTempView("dv")
dv.show(5)

+------------------+-----------------+-----------------+----+--------------------+--------------------+--------------------+-------------------+------------------+
|              time|         position|original_velocity|pump|         neural_data|           file_name|                  dt|                 dx|          velocity|
+------------------+-----------------+-----------------+----+--------------------+--------------------+--------------------+-------------------+------------------+
|0.9684967780795763|167.8834395309168|15.33076223119325| 0.0|[0.00292782419273...|ID17905/Day5/Data...|0.031241831550954058|0.11094945690160785|3.5513108993195277|
|0.9997386096305304|168.0412270034912|13.94933848866475| 0.0|[0.00290607741044...|ID17905/Day5/Data...|0.031241831550954058|0.15778747257439818| 5.050519279481222|
| 1.030980441181484|168.2425834116714|12.65062656640945| 0.0|[0.00314377625181...|ID17905/Day5/Data...|0.031241831550953614|0.20135640818020306| 6.445089746156606|
| 1.062222272732

windowSpec: WindowSpec = org.apache.spark.sql.expressions.WindowSpec@177fc108
dv: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [time: double, position: double ... 7 more fields]

In [6]:
spark.sql("select distinct(pump) from dv limit 5").show(5)

+----+
|pump|
+----+
| 0.0|
| 1.0|
+----+



In [7]:
import java.io.File

val directory = new File("/main/data/transformed")
val file = new File(directory, "test.txt")
file.createNewFile()


java.io.IOException: Permission denied

In [9]:
dv.write.partitionBy("file_name").mode("overwrite").parquet("/main/data/transformed/")

org.apache.spark.SparkException: Unable to clear output directory file:/main/data/transformed prior to writing to it.